引用

In [1]:
import pandas as pd
import os
from collections import Counter

讀取資料

In [2]:
total_data=pd.read_excel(f"./病房人數.xlsx" ,sheet_name ="開刀資料")
total_data

,手術日期,護理站,成本中心,人數
0,2018-01-02,D15,DLN5,1
1,2018-01-03,D15,DLN5,6
2,2018-01-04,D15,DLN5,6
3,2018-01-05,D15,DLN5,4
4,2018-01-06,D15,DLN5,2
...,...,...,...,...
3041,2019-09-27,D22,DLN2,1
3042,2020-03-06,D23,DLN3,1
3043,2020-04-16,D23,DLN3,1
3044,2020-08-04,D23,DLN3,1


In [3]:
# 依照護理站分群
count_class = Counter(total_data["成本中心"])
count_class = count_class.keys()
# 將資料分組
group = total_data.groupby(['成本中心'])
for type in group.groups.keys():
    print(type)

DLI1
DLN2
DLN3
DLN5
DLN6
DLN8
DLRI


In [4]:
def take_data(data,n_day):
    data_copy = data.copy()
    # 總共要提取幾次
    for num in range(n_day, len(data_copy),1):
#         每次提取n筆
        for time in range(1,n_day+1,1):
            before_day = data_copy.loc[num-time:num-time,"該週總人數"]
            print(before_day)
#             print(type(before_day))
            data_copy.loc[num:num,f'前{time}週人數'] = before_day.values[0]

    return data_copy


五病房資料

In [5]:
room5 = total_data[total_data['成本中心'] == 'DLN5'] 
room5
# 先計算每週總人數、平均人數、最多人數
room5['手術日期'] =  pd.to_datetime(room5['手術日期'])
room5['週'] = room5['手術日期'].dt.isocalendar().week
room5['年'] = room5['手術日期'].dt.isocalendar().year
room5['開刀總人數'] = 0
# 先移除第53週
room5 = room5.drop(room5[room5['週'] == 53].index).reset_index(drop=True)
print()
# 計算一週總人數
for year in range(0,6,1):
    for num in range(1, 53,1):
        temp = room5[(room5['年'] == 2018+year) & (room5['週'] == num)]
        if (year == 4) & (num == 5):
            temp = room5.head(1)
            temp['年']  = 2022.
            temp['週'] = 5
            temp['人數'] = 0
            room5 = room5.append(temp)
        people_sum = temp['人數'].sum()
        people_max = temp['人數'].max()
        room5.loc[(room5['年'] == 2018+year) & (room5['週'] == num),'開刀總人數'] = people_sum
        room5.loc[(room5['年'] == 2018+year) & (room5['週'] == num),'開刀最多人'] = people_max
# 按照年、週排序
room5 = room5.sort_values(['年', '週'], ascending=True)
            
columns = ['週','年']     
room5.drop_duplicates(subset=columns, keep='last', inplace=True)
room5 = room5.reset_index(drop=True)

for num in range(1,len(room5),1):
        before_day = room5.loc[num-1:num-1,"開刀總人數"]
        before_day_max = room5.loc[num-1:num-1,"開刀最多人"]
        print(before_day)
#             print(type(before_day))
        room5.loc[num:num,f'前{1}週開刀人數'] = before_day.values[0]
        room5.loc[num:num,f'前{1}週開刀最多人'] = before_day_max.values[0]

# 計算前一週病人平均數

for year in range(0,6,1):
    for num in range(1, 53,1):
        temp = room5[(room5['年'] == 2018+year) & (room5['週'] == num)]
        people_avg = (temp['前1週開刀人數'].values[0]/7).round(decimals = 2)
        room5.loc[(room5['年'] == 2018+year) & (room5['週'] == num),'前1週開刀平均人數'] = people_avg



C:\Users\IDSL\AppData\Local\Temp\ipykernel_15868\3205276225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room5['手術日期'] =  pd.to_datetime(room5['手術日期'])
C:\Users\IDSL\AppData\Local\Temp\ipykernel_15868\3205276225.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room5['週'] = room5['手術日期'].dt.isocalendar().week
C:\Users\IDSL\AppData\Local\Temp\ipykernel_15868\3205276225.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

0    19
Name: 開刀總人數, dtype: int64
1    31
Name: 開刀總人數, dtype: int64
2    34
Name: 開刀總人數, dtype: int64
3    29
Name: 開刀總人數, dtype: int64
4    27
Name: 開刀總人數, dtype: int64
5    23
Name: 開刀總人數, dtype: int64
6    22
Name: 開刀總人數, dtype: int64
7    14
Name: 開刀總人數, dtype: int64
8    38
Name: 開刀總人數, dtype: int64
9    32
Name: 開刀總人數, dtype: int64
10    35
Name: 開刀總人數, dtype: int64
11    27
Name: 開刀總人數, dtype: int64
12    34
Name: 開刀總人數, dtype: int64
13    24
Name: 開刀總人數, dtype: int64
14    43
Name: 開刀總人數, dtype: int64
15    32
Name: 開刀總人數, dtype: int64
16    26
Name: 開刀總人數, dtype: int64
17    35
Name: 開刀總人數, dtype: int64
18    26
Name: 開刀總人數, dtype: int64
19    34
Name: 開刀總人數, dtype: int64
20    33
Name: 開刀總人數, dtype: int64
21    30
Name: 開刀總人數, dtype: int64
22    29
Name: 開刀總人數, dtype: int64
23    32
Name: 開刀總人數, dtype: int64
24    24
Name: 開刀總人數, dtype: int64
25    29
Name: 開刀總人數, dtype: int64
26    47
Name: 開刀總人數, dtype: int64
27    41
Name: 開刀總人數, dtype: int64
28    28
Name: 開刀總人數, dtype: i

IndexError: index 0 is out of bounds for axis 0 with size 0

In [6]:
room5.tail(60)

,手術日期,護理站,成本中心,人數,週,年,開刀總人數,開刀最多人,前1週開刀人數,前1週開刀最多人,前1週開刀平均人數
209,2022-01-16,D15,DLN5,1,2,2022.0,34,11.0,33.0,8.0,4.71
210,2022-01-21,D15,DLN5,2,3,2022.0,28,11.0,34.0,11.0,4.86
211,2022-01-27,D15,DLN5,6,4,2022.0,23,8.0,28.0,11.0,4.00
212,2018-01-02,D15,DLN5,0,5,2022.0,0,0.0,23.0,8.0,3.29
213,2022-02-13,D15,DLN5,2,6,2022.0,37,11.0,0.0,0.0,0.00
214,2022-02-20,D15,DLN5,1,7,2022.0,40,11.0,37.0,11.0,5.29
215,2022-02-27,D15,DLN5,3,8,2022.0,41,10.0,40.0,11.0,5.71
216,2022-03-06,D15,DLN5,2,9,2022.0,37,11.0,41.0,10.0,5.86
217,2022-03-13,D15,DLN5,1,10,2022.0,45,14.0,37.0,11.0,5.29
218,2022-03-20,D15,DLN5,2,11,2022.0,42,15.0,45.0,14.0,6.43


In [7]:
room5.to_csv(f'./五病房開刀人數.csv',index=False, encoding='utf_8_sig') 